In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge, Lasso

import cvxpy as cp

# Sanity check for rotation results. Code for Table E.1.

In [ ]:
X = pd.read_csv('./data/nsv_psid_X.csv', index_col=0)
T = pd.read_csv('./data/nsv_psid_T.csv', index_col=0)
Y = pd.read_csv('./data/nsv_psid_Y.csv', index_col=0)

In [ ]:
Y = np.array(Y).reshape(-1)
T = np.array(T).reshape(-1)
X = np.array(X)

### separate control and treated groups

In [4]:
Xp = X[T==0]
Xq = X[T==1]

yp = Y[T==0]

In [5]:
Xpc = Xp - Xp.mean(axis=0)
ypc = yp - yp.mean()

In [6]:
Xqc = (Xq - Xp.mean(axis=0))
Xqb = Xqc.mean(axis=0)

In [7]:
lam = 80

In [8]:
betahat = Ridge(alpha=lam, fit_intercept=False).fit(Xpc,ypc).coef_

In [ ]:
def ell2(delta,Xp,Xq):

    N=Xp.shape[0]
    M=Xp.shape[1]

    rho = cp.Variable(M)
    w = Xp@rho

    prob = cp.Problem(cp.Minimize( cp.sum_squares(w) - 2*rho@Xq + delta*cp.sum_squares(rho) ))
    
    optimal_value = prob.solve()
    return(rho.value)

In [ ]:
delta = 10
rhohat = ell2(delta, Xpc, Xqb)
what = Xpc@rhohat
qhat = what@Xpc
beta_ols = np.linalg.pinv(Xpc.T@Xpc)@Xpc.T@ypc
aug_point = Xqb@betahat + what@(ypc-Xpc@betahat)

In [ ]:
# PROPER beta aug
d = Xpc.shape[1]
A = np.linalg.inv(Xpc.T@Xpc + delta*np.eye(d)) @ (Xpc.T@Xpc)
beta_aug_proper = A@beta_ols + (np.eye(d) - A)@betahat
# IMPROPER beta aug
a = qhat/Xqb
beta_aug_improper = a*beta_ols + (1-a)*betahat

#### now compute rotated version

In [25]:
evls,evec = np.linalg.eigh(Xpc.T@Xpc)

In [26]:
qhat_rotated = what@(Xpc@evec)

Xqb_rotated = Xqb@evec

a_rotated = qhat_rotated/Xqb_rotated

In [27]:
beta_aug_rotated = evec@(a_rotated* (evec.T@beta_ols) + (1-a_rotated) *(evec.T@betahat))

In [29]:
np.linalg.norm(beta_aug_rotated)

18615.761538449882

In [30]:
np.linalg.norm(beta_aug_proper)

18615.761097176397

In [31]:
np.linalg.norm(beta_ols)

44698074.06257617

In [32]:
np.linalg.norm(beta_aug_improper)

45860522.93505297

In [35]:
D = np.diag(evls)

In [46]:
a_theory = evls/(evls+delta)

# Very quick sanity check with non-ridge outcome model

In [ ]:
beta_lasso = Lasso(alpha=80, fit_intercept=False).fit(Xpc, ypc).coef_
beta_aug_proper_lasso = A@beta_ols + (np.eye(d) - A)@beta_lasso
beta_aug_proper_rerot = A@beta_ols + (np.eye(d) - A)@evec@evec.T@beta_lasso
beta_aug_rot_lasso = evec@(a_theory*(evec.T@beta_ols) + (1-a_theory)*(evec.T@beta_lasso))

In [61]:
np.linalg.norm(beta_lasso)

12429.748270392154

In [62]:
np.linalg.norm(beta_aug_proper_lasso)

20911.724802910263

In [64]:
np.linalg.norm(beta_aug_proper_rerot)

20911.72480291026

In [66]:
np.linalg.norm(beta_aug_rot_lasso)

20911.72480284092

In [68]:
np.linalg.norm(beta_ols)

44698074.06257617